# Lasso回归 (Lasso Regression)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解Lasso回归的原理和L1正则化
- 掌握Lasso的稀疏性特征（特征选择功能）
- 学会使用Scikit-learn实现Lasso回归
- 比较Lasso与Ridge回归的区别
- 理解何时使用Lasso进行特征选择

## 📋 前置知识

- 线性回归基础
- 岭回归 (Ridge Regression)
- 过拟合与正则化概念

## ⏱️ 预计时间

25-35分钟

---

## 📖 第1部分：理论背景

### 什么是Lasso回归？

**Lasso（Least Absolute Shrinkage and Selection Operator）回归**是一种带有**L1正则化**的线性回归模型。它的独特之处在于能够将某些参数精确地收缩到0，从而实现**自动特征选择**。

### 数学模型

#### Lasso回归的损失函数

$$J(\theta) = \frac{1}{2m}||X\theta - y||^2 + \alpha\sum_{j=1}^{n}|\theta_j|$$

其中：
- $\sum_{j=1}^{n}|\theta_j|$ 是L1范数（不包括偏置项$\theta_0$）
- $\alpha$ 是正则化强度

### L1 vs L2 正则化

| 特性 | L1 (Lasso) | L2 (Ridge) |
|------|------------|------------|
| 正则化项 | $\alpha\sum|\theta_j|$ | $\alpha\sum\theta_j^2$ |
| 几何形状 | 菱形 | 圆形 |
| 稀疏性 | 产生稀疏解 | 不产生稀疏解 |
| 特征选择 | ✅ 自动特征选择 | ❌ 保留所有特征 |
| 求解方法 | 需要特殊算法 | 有闭式解 |

### 为什么L1能产生稀疏解？

从几何角度理解：
- L1正则化的约束区域是菱形
- 损失函数的等高线与菱形的顶点相交的概率更高
- 菱形的顶点恰好在坐标轴上，意味着某些参数为0

> 💡 **关键洞察**: Lasso回归不仅能防止过拟合，还能自动进行特征选择，将不重要特征的系数置为0。

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

# 数值计算
import numpy as np

# 可视化
import matplotlib.pyplot as plt

# 机器学习
from sklearn.linear_model import Lasso, LassoCV, Ridge, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 数据参数
N_SAMPLES = 100
NOISE_LEVEL = 1.0

# 可视化配置
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 库导入和配置完成")

### 步骤2: 生成数据

In [ ]:
# ============================================================
# 生成非线性数据
# ============================================================

# 生成特征X
X = 6 * np.random.rand(N_SAMPLES, 1) - 3

# 生成目标y：y = 0.5x² + x + 2 + 噪声
y = 0.5 * X**2 + X + 2 + NOISE_LEVEL * np.random.randn(N_SAMPLES, 1)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_SEED
)

print(f"✓ 数据生成完成")
print(f"  训练集: {len(X_train)} 样本")
print(f"  测试集: {len(X_test)} 样本")

### 步骤3: 基础Lasso回归

In [ ]:
# ============================================================
# 基础Lasso回归
# ============================================================

# 创建Lasso回归模型
# alpha: 正则化强度
# max_iter: 最大迭代次数（Lasso使用坐标下降法求解）
lasso_reg = Lasso(alpha=0.1, max_iter=10000)

# 训练模型
lasso_reg.fit(X, y.ravel())

# 预测
y_pred = lasso_reg.predict(X)

# 评估
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("基础Lasso回归结果:")
print(f"  截距: {lasso_reg.intercept_:.4f}")
print(f"  系数: {lasso_reg.coef_}")
print(f"  MSE: {mse:.4f}")
print(f"  R²: {r2:.4f}")

---

## 🔬 第3部分：Lasso的特征选择能力

### 创建高维数据演示特征选择

In [ ]:
# ============================================================
# 创建高维数据：只有少数特征是重要的
# ============================================================

# 生成20个特征，但只有前3个是有用的
n_features = 20
n_informative = 3

# 生成随机特征
X_high = np.random.randn(N_SAMPLES, n_features)

# 真实系数：只有前3个非零
true_coefs = np.zeros(n_features)
true_coefs[:n_informative] = [3.0, -2.0, 1.5]

# 生成目标值
y_high = X_high @ true_coefs + 0.5 * np.random.randn(N_SAMPLES)

# 划分数据
X_high_train, X_high_test, y_high_train, y_high_test = train_test_split(
    X_high, y_high, test_size=0.3, random_state=RANDOM_SEED
)

print(f"✓ 高维数据生成完成")
print(f"  特征数量: {n_features}")
print(f"  有效特征: {n_informative}")
print(f"  真实系数: {true_coefs[:5]}... (只显示前5个)")

### 比较Lasso、Ridge和普通线性回归

In [ ]:
# ============================================================
# 三种模型对比
# ============================================================

# 创建三种模型
linear = LinearRegression()
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.1, max_iter=10000)

# 训练模型
linear.fit(X_high_train, y_high_train)
ridge.fit(X_high_train, y_high_train)
lasso.fit(X_high_train, y_high_train)

# 计算测试误差
linear_pred = linear.predict(X_high_test)
ridge_pred = ridge.predict(X_high_test)
lasso_pred = lasso.predict(X_high_test)

# 可视化系数
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 真实系数
axes[0, 0].bar(range(n_features), true_coefs, color='green', alpha=0.7)
axes[0, 0].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axes[0, 0].set_xlabel('特征索引')
axes[0, 0].set_ylabel('系数值')
axes[0, 0].set_title('真实系数', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 普通线性回归
axes[0, 1].bar(range(n_features), linear.coef_, color='blue', alpha=0.7)
axes[0, 1].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axes[0, 1].set_xlabel('特征索引')
axes[0, 1].set_ylabel('系数值')
axes[0, 1].set_title(f'线性回归 (MSE: {mean_squared_error(y_high_test, linear_pred):.4f})', 
                     fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 岭回归
axes[1, 0].bar(range(n_features), ridge.coef_, color='orange', alpha=0.7)
axes[1, 0].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axes[1, 0].set_xlabel('特征索引')
axes[1, 0].set_ylabel('系数值')
axes[1, 0].set_title(f'岭回归 (MSE: {mean_squared_error(y_high_test, ridge_pred):.4f})', 
                     fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Lasso回归
axes[1, 1].bar(range(n_features), lasso.coef_, color='red', alpha=0.7)
axes[1, 1].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axes[1, 1].set_xlabel('特征索引')
axes[1, 1].set_ylabel('系数值')
axes[1, 1].set_title(f'Lasso回归 (MSE: {mean_squared_error(y_high_test, lasso_pred):.4f})', 
                     fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 统计非零系数数量
print("\n系数分析:")
print(f"  真实非零系数数量: {np.sum(true_coefs != 0)}")
print(f"  线性回归非零系数: {np.sum(np.abs(linear.coef_) > 1e-10)}")
print(f"  岭回归非零系数: {np.sum(np.abs(ridge.coef_) > 1e-10)}")
print(f"  Lasso非零系数: {np.sum(np.abs(lasso.coef_) > 1e-10)}")

### α值对Lasso稀疏性的影响

In [ ]:
# ============================================================
# 不同α值的稀疏性分析
# ============================================================

alphas = np.logspace(-3, 1, 50)
n_nonzero_coefs = []
test_mses = []
coef_paths = []

for alpha in alphas:
    lasso_temp = Lasso(alpha=alpha, max_iter=10000)
    lasso_temp.fit(X_high_train, y_high_train)
    
    # 记录非零系数数量
    n_nonzero_coefs.append(np.sum(np.abs(lasso_temp.coef_) > 1e-10))
    
    # 记录测试MSE
    y_pred_temp = lasso_temp.predict(X_high_test)
    test_mses.append(mean_squared_error(y_high_test, y_pred_temp))
    
    # 记录系数路径
    coef_paths.append(lasso_temp.coef_.copy())

coef_paths = np.array(coef_paths)

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 左图：非零系数数量
axes[0].plot(alphas, n_nonzero_coefs, 'b-', linewidth=2)
axes[0].axhline(y=n_informative, color='green', linestyle='--', label=f'真实非零数={n_informative}')
axes[0].set_xscale('log')
axes[0].set_xlabel('α (正则化强度)')
axes[0].set_ylabel('非零系数数量')
axes[0].set_title('Lasso稀疏性', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 中图：测试MSE
axes[1].plot(alphas, test_mses, 'r-', linewidth=2)
best_idx = np.argmin(test_mses)
axes[1].axvline(x=alphas[best_idx], color='green', linestyle='--', 
                label=f'最佳α={alphas[best_idx]:.4f}')
axes[1].scatter([alphas[best_idx]], [test_mses[best_idx]], color='green', s=100, zorder=5)
axes[1].set_xscale('log')
axes[1].set_xlabel('α (正则化强度)')
axes[1].set_ylabel('测试MSE')
axes[1].set_title('测试误差随α变化', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 右图：系数路径
for i in range(n_features):
    if i < n_informative:
        axes[2].plot(alphas, coef_paths[:, i], linewidth=2, label=f'θ{i} (重要)')
    else:
        axes[2].plot(alphas, coef_paths[:, i], linewidth=0.5, alpha=0.5)
        
axes[2].set_xscale('log')
axes[2].set_xlabel('α (正则化强度)')
axes[2].set_ylabel('系数值')
axes[2].set_title('Lasso系数路径', fontsize=12, fontweight='bold')
axes[2].legend(loc='upper right', fontsize=8)
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='black', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

print(f"\n最佳α值: {alphas[best_idx]:.4f}")
print(f"对应非零系数数量: {n_nonzero_coefs[best_idx]}")
print(f"对应测试MSE: {test_mses[best_idx]:.4f}")

---

## 📊 第4部分：交叉验证选择α

### 使用LassoCV自动选择最佳α

In [ ]:
# ============================================================
# 使用LassoCV自动选择α
# ============================================================

# LassoCV自动执行交叉验证选择最佳alpha
lasso_cv = LassoCV(
    alphas=np.logspace(-4, 1, 100),  # 候选alpha值
    cv=5,                             # 5折交叉验证
    max_iter=10000,
    random_state=RANDOM_SEED
)

lasso_cv.fit(X_high_train, y_high_train)

# 结果
print("LassoCV结果:")
print(f"  最佳α: {lasso_cv.alpha_:.6f}")
print(f"  非零系数数量: {np.sum(np.abs(lasso_cv.coef_) > 1e-10)}")

# 测试性能
y_cv_pred = lasso_cv.predict(X_high_test)
print(f"  测试MSE: {mean_squared_error(y_high_test, y_cv_pred):.4f}")
print(f"  测试R²: {r2_score(y_high_test, y_cv_pred):.4f}")

# 可视化交叉验证结果
plt.figure(figsize=(10, 5))
plt.semilogx(lasso_cv.alphas_, lasso_cv.mse_path_.mean(axis=1), 'b-', linewidth=2, label='平均MSE')
plt.fill_between(lasso_cv.alphas_, 
                 lasso_cv.mse_path_.mean(axis=1) - lasso_cv.mse_path_.std(axis=1),
                 lasso_cv.mse_path_.mean(axis=1) + lasso_cv.mse_path_.std(axis=1),
                 alpha=0.2)
plt.axvline(x=lasso_cv.alpha_, color='red', linestyle='--', 
            label=f'最佳α = {lasso_cv.alpha_:.6f}')
plt.xlabel('α (正则化强度)')
plt.ylabel('交叉验证MSE')
plt.title('LassoCV: 交叉验证选择最佳α', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 分析选中的特征

In [ ]:
# ============================================================
# 分析Lasso选中的特征
# ============================================================

# 获取非零系数的特征
selected_features = np.where(np.abs(lasso_cv.coef_) > 1e-10)[0]

print("Lasso特征选择结果:")
print(f"  选中的特征索引: {selected_features}")
print(f"  真实重要特征索引: [0, 1, 2]")
print(f"\n系数对比:")
print(f"  {'特征':<8} {'Lasso系数':>12} {'真实系数':>12}")
print(f"  {'-'*35}")
for i in selected_features:
    print(f"  {f'特征{i}':<8} {lasso_cv.coef_[i]:>12.4f} {true_coefs[i]:>12.4f}")

---

## 🔄 第5部分：多项式回归中的Lasso

让我们在多项式回归问题中比较Lasso和Ridge：

In [ ]:
# ============================================================
# 多项式回归中的Lasso vs Ridge
# ============================================================

# 使用少量训练数据
X_small = X[:20]
y_small = y[:20]

# 多项式次数
POLY_DEGREE = 10

# 生成绘图用的平滑数据
X_line = np.linspace(-3, 3, 100).reshape(-1, 1)

# 创建模型管道
models = {
    '无正则化': Pipeline([
        ('poly', PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)),
        ('scaler', StandardScaler()),
        ('reg', LinearRegression())
    ]),
    'Ridge (α=1)': Pipeline([
        ('poly', PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)),
        ('scaler', StandardScaler()),
        ('reg', Ridge(alpha=1.0))
    ]),
    'Lasso (α=0.01)': Pipeline([
        ('poly', PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)),
        ('scaler', StandardScaler()),
        ('reg', Lasso(alpha=0.01, max_iter=10000))
    ])
}

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 真实曲线
y_true = 0.5 * X_line**2 + X_line + 2

for ax, (name, model) in zip(axes, models.items()):
    # 训练
    model.fit(X_small, y_small.ravel())
    
    # 预测
    y_line_pred = model.predict(X_line)
    y_small_pred = model.predict(X_small)
    
    # 计算MSE
    train_mse = mean_squared_error(y_small, y_small_pred)
    
    # 获取非零系数数量
    coefs = model.named_steps['reg'].coef_
    n_nonzero = np.sum(np.abs(coefs) > 1e-10)
    
    # 绘图
    ax.scatter(X_small, y_small, color='blue', alpha=0.6, s=50, label='训练数据')
    ax.plot(X_line, y_line_pred, 'r-', linewidth=2, label='预测曲线')
    ax.plot(X_line, y_true, 'g--', linewidth=2, label='真实曲线')
    ax.set_xlabel('X')
    ax.set_ylabel('y')
    ax.set_title(f'{name}\nMSE: {train_mse:.2f}, 非零系数: {n_nonzero}')
    ax.set_ylim(-5, 15)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 📝 总结

### 关键要点

1. ✅ **Lasso回归**使用L1正则化，能够产生**稀疏解**
2. ✅ Lasso可以自动进行**特征选择**，将不重要特征的系数置为0
3. ✅ L1正则化的几何形状（菱形）导致了稀疏性
4. ✅ 使用LassoCV可以自动选择最佳正则化参数

### Lasso vs Ridge 选择指南

| 场景 | 推荐模型 |
|------|----------|
| 需要特征选择 | Lasso |
| 特征数量 > 样本数量 | Lasso |
| 多重共线性严重 | Ridge |
| 所有特征都重要 | Ridge |
| 不确定 | Elastic Net (L1+L2) |

### 注意事项

> ⚠️ **重要**: Lasso在特征高度相关时，倾向于只选择其中一个特征

> 💡 **建议**: 如果既需要特征选择又担心丢失相关特征，考虑使用Elastic Net

### 下一步学习

- 📘 Elastic Net (弹性网络)
- 📙 特征选择方法比较
- 📕 稀疏模型的应用

### 练习题

1. 尝试在真实数据集上使用Lasso进行特征选择
2. 比较不同α值下Lasso选择的特征
3. 使用Lasso系数的绝对值作为特征重要性指标

## 📚 参考资料

- [Scikit-learn: Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
- [Scikit-learn: LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html)
- Tibshirani, R. (1996). "Regression Shrinkage and Selection via the Lasso"